In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [10]:
def get_page(link = '/soccer'):
    page_link = 'http://www.betexplorer.com'+link    
    try:
        # fetch the content from url
        page_response = requests.get(page_link, timeout=5)
        # parse html
        page_content = BeautifulSoup(page_response.content, "html.parser")
    except:
        #caso a página não responda, espera por 5min e tenta novamente
        time.sleep(300)
        page_response = requests.get(page_link, timeout=60)
        # parse html
        page_content = BeautifulSoup(page_response.content, "html.parser")
    return page_content


def scrapGroupPhase(page, page_link, tournament, season):
    #df_goals = pd.DataFrame(columns=['Match_Id','Team_Id', 'Time','Player_Name']) #para coleta d
    df = pd.DataFrame(columns=['Tournament', 'Season','Round','Home', 'Away','Score','Bet_Home','Bet_Drawn','Bet_Away','Game_Date','Game_Details','Fase'])
    #div = page_content.find_all('ul', attrs = {'class' : 'list-tabs list-tabs--secondary'})
    th_position = 0
    td_position = 0
    data = []
    for content in page:
        for li in content.find_all('li'):            
            fase = li.text
            link = li.select('a')[0].attrs['href']
            page_content = get_page(page_link+link)
            table = page_content.find_all('table')[0]
            for row in table.find_all(['td', 'th']):
                if row.name == 'th' and th_position == 0:
                    rodada = row.text
                    data = [tournament, season, rodada]
                    th_position = 1
                elif row.name == 'td':
                    if th_position == 0:
                        rodada = fase
                        data = [tournament, season, rodada]
                        th_position = 1
                    if td_position == 0:
                        data.append(row.text.split(' - ')[0])
                        data.append(row.text.split(' - ')[1])
                        for a in row.find_all('a', href=True):
                            link = a['href']
                            #df_goals = scrapGoals(len(df),[row.text.split(' - ')[0],row.text.split(' - ')[1]],link, df_goals)
                    elif 2 <= td_position <= 4:
                        try:
                            data.append(row.select('span')[2].attrs['data-odd'])
                        except:
                            try: 
                                data.append(row.attrs['data-odd'])
                            except:
                                data.append(row.text)                    
                    else:
                        data.append(row.text)
                    td_position+=1       
                    if td_position == 6:
                        data.append(link)
                        data.append(fase)
                        df.loc[len(df)] = data
                        data = [tournament, season, rodada]
                        th_position = 0
                        td_position = 0
    #return df, df_goals
    return df

In [11]:
def scrapChampionship(page_content, tournament, season):
    #df_goals = pd.DataFrame(columns=['Match_Id','Team_Id', 'Time','Player_Name'])
    df = pd.DataFrame(columns=['Tournament', 'Season','Round','Home', 'Away','Score','Bet_Home','Bet_Drawn','Bet_Away','Game_Date','Game_Details','Fase'])
    th_position = 0
    td_position = 0
    data = []
    table = page_content.find_all('table')[0]
    for row in table.find_all(['td', 'th']):
        if row.name == 'th' and th_position == 0:
            rodada = row.text
            data = [tournament, season, rodada]
            th_position = 1
        elif row.name == 'td':
            if td_position == 0:
                data.append(row.text.split(' - ')[0])
                data.append(row.text.split(' - ')[1])
                for a in row.find_all('a', href=True):
                    link = a['href']
                    #df_goals = scrapGoals(len(df),[row.text.split(' - ')[0],row.text.split(' - ')[1]],link, df_goals)
            elif 2 <= td_position <= 4:
                try:
                    data.append(row.select('span')[2].attrs['data-odd'])
                except:
                    try: 
                        data.append(row.attrs['data-odd'])
                    except:
                        data.append(row.text)                    
            else:
                data.append(row.text)
            td_position+=1       
            if td_position == 6:
                data.append(link)
                data.append('')
                df.loc[len(df)] = data
                data = [tournament, season, rodada]
                th_position = 0
                td_position = 0
    #return df, df_goals
    return df

def scrapGoals(match_id, team_id, page_link, df_goals):
    page_content = get_page(page_link)
    for ul in page_content.find_all('ul', attrs = {'class' : 'list-details list-details--shooters'}):
        i = 0
        for li in ul.findAll('li'):
            if len(li) > 1:
                for tr in li.findAll('tr'):
                    #home team
                    if i == 0:
                        df_goals.loc[len(df_goals)] = [match_id, team_id[0], tr.text.split('.')[0],tr.text.split('.')[1]]
                    #away team
                    else:
                        df_goals.loc[len(df_goals)] = [match_id, team_id[1], tr.text.split('.')[0],tr.text.split('.')[1]]
            i = 1

    return df_goals

In [12]:
df_tournaments_links = pd.read_excel('data/Tournament Links.xlsx')
log_columns = list(df_tournaments_links.columns)
log_columns.append('Function')
log_columns.append('Error')
log = pd.DataFrame(columns= log_columns)

In [13]:
#seleciona torneios a serem coletados
df_tournaments_links = df_tournaments_links[(df_tournaments_links.Country >= 'Sweden')]
df_tournaments_links

,Country,Tournament,Season,Link
5555,Sweden,Allsvenskan,2018,/soccer/sweden/allsvenskan/
5556,Sweden,Superettan,2018,/soccer/sweden/superettan/
5557,Sweden,Division 1 - Norra,2018,/soccer/sweden/division-1-norra/
5558,Sweden,Division 1 - Södra,2018,/soccer/sweden/division-1-sodra/
5559,Sweden,Division 2 - Norrland,2018,/soccer/sweden/division-2-norrland/
5560,Sweden,Division 2 - Norra Svealand,2018,/soccer/sweden/division-2-norra-svealand/
5561,Sweden,Division 2 - Västra Götaland,2018,/soccer/sweden/division-2-vastra-gotaland/
5562,Sweden,Division 2 - Södra Svealand,2018,/soccer/sweden/division-2-sodra-svealand/
5563,Sweden,Division 2 - Norra Götaland,2018,/soccer/sweden/division-2-norra-gotaland/
5564,Sweden,Division 2 - Östra Götaland,2018,/soccer/sweden/division-2-ostra-gotaland/


In [14]:
%%time
current_country = df_tournaments_links.loc[df_tournaments_links.index.min()].Country
df_country_competitions = pd.DataFrame(columns=['Tournament', 'Season','Round','Home', 'Away','Score','Bet_Home','Bet_Drawn','Bet_Away','Game_Date','Game_Details','Fase'])
for row in df_tournaments_links.values:
    row = list(row)
    page_link = row[3]
    try:
        page_content = get_page(page_link+'results')
    except:
        row.append('get_page')
        row.append('Timeout')
        log.loc[len(log)] = row
        break
    div = page_content.find_all('ul', attrs = {'class' : 'list-tabs list-tabs--secondary'})
    try:
        if len(div) > 0:
            row.append('scrapGroupPhase')
            df = scrapGroupPhase(div, page_link, row[1], row[2])
            #df, df_goals = scrapGroupPhase(div, page_link)
        else:
            row.append('scrapChampionship')
            #df, df_goals = scrapChampionship(page_content)
            df = scrapChampionship(page_content, row[1], row[2])        
        #if len(df_goals) > 0:
            #df_goals.to_excel('data/'+row[1]+' - '+row[2].split('/')[0]+' - Goals.xlsx', index=False)
        if row[0] == current_country:
            df_country_competitions = df_country_competitions.append(df)
        else:
            df_country_competitions.to_excel('data/'+current_country+'.xlsx', index=False)
            df_country_competitions = df
            current_country = row[0]
    except Exception as e:
        row.append(str(e))
        log.loc[len(log)] = row
        print(row)
df_country_competitions.to_excel('data/'+current_country+'.xlsx', index=False)
log.to_csv('data/log.csv', encoding=None, index=False)

['Sweden', 'Allsvenskan', '2018', '/soccer/sweden/allsvenskan/', 'scrapChampionship', 'list index out of range']
['Sweden', 'Superettan', '2018', '/soccer/sweden/superettan/', 'scrapChampionship', 'list index out of range']
['Sweden', 'Division 1 - Norra', '2018', '/soccer/sweden/division-1-norra/', 'scrapChampionship', 'list index out of range']
['Sweden', 'Division 1 - Södra', '2018', '/soccer/sweden/division-1-sodra/', 'scrapChampionship', 'list index out of range']
['Sweden', 'Division 2 - Norrland', '2018', '/soccer/sweden/division-2-norrland/', 'scrapChampionship', 'list index out of range']
['Sweden', 'Division 2 - Norra Svealand', '2018', '/soccer/sweden/division-2-norra-svealand/', 'scrapChampionship', 'list index out of range']
['Sweden', 'Division 2 - Västra Götaland', '2018', '/soccer/sweden/division-2-vastra-gotaland/', 'scrapChampionship', 'list index out of range']
['Sweden', 'Division 2 - Södra Svealand', '2018', '/soccer/sweden/division-2-sodra-svealand/', 'scrapChampi

['United Arab Emirates', 'FA Cup', '2015/2016', '/soccer/united-arab-emirates/fa-cup/', 'scrapChampionship', 'cannot set a row with mismatched columns']
['Uruguay', 'Primera Division', '2018', '/soccer/uruguay/primera-division/', 'scrapGroupPhase', 'list index out of range']
['USA', 'MLS', '2018', '/soccer/usa/mls/', 'scrapChampionship', 'cannot set a row with mismatched columns']
['USA', 'USL', '2018', '/soccer/usa/usl/', 'scrapChampionship', 'cannot set a row with mismatched columns']
['USA', 'Carolina Challenge Cup', '2018', '/soccer/usa/carolina-challenge-cup/', 'scrapChampionship', 'cannot set a row with mismatched columns']
['USA', 'NWSL Women', '2018', '/soccer/usa/nwsl-women/', 'scrapChampionship', 'cannot set a row with mismatched columns']
['USA', 'Carolina Challenge Cup', '2017', '/soccer/usa/carolina-challenge-cup-2017/', 'scrapChampionship', 'cannot set a row with mismatched columns']
['USA', 'Carolina Challenge Cup', '2015', '/soccer/usa/carolina-challenge-cup-2015/', 'sc

['World', 'Friendly International', '2014', '/soccer/world/friendly-international-2014/', 'scrapChampionship', 'cannot set a row with mismatched columns']
['World', 'Club Friendly', '2014', '/soccer/world/club-friendly-2014/', 'scrapChampionship', 'cannot set a row with mismatched columns']
['World', 'La Manga Cup', '2014', '/soccer/world/la-manga-cup-2014/', 'scrapChampionship', 'cannot set a row with mismatched columns']
['World', 'Friendly International Women', '2014', '/soccer/world/friendly-international-women-2014/', 'scrapChampionship', 'cannot set a row with mismatched columns']
['World', 'Friendly International', '2013', '/soccer/world/friendly-international-2013/', 'scrapChampionship', 'cannot set a row with mismatched columns']
['World', 'Club Friendly', '2013', '/soccer/world/club-friendly-2013/', 'scrapChampionship', 'cannot set a row with mismatched columns']
['World', 'La Manga Cup', '2013', '/soccer/world/la-manga-cup-2013/', 'scrapChampionship', 'cannot set a row with 

['World', 'Kirin Cup (Japan)', '2007', '/soccer/world/kirin-cup-japan-2007/', 'scrapChampionship', 'cannot set a row with mismatched columns']
['World', 'Barclays Asia Trophy', '2007', '/soccer/world/barclays-asia-trophy-2007/', 'scrapChampionship', 'cannot set a row with mismatched columns']
['World', 'Chicago Trophy', '2007', '/soccer/world/chicago-trophy-2007/', 'scrapChampionship', 'cannot set a row with mismatched columns']
['World', 'La Manga Cup', '2007', '/soccer/world/la-manga-cup-2007/', 'scrapChampionship', 'cannot set a row with mismatched columns']
['World', 'LG Amsterdam', '2007', '/soccer/world/lg-amsterdam-2007/', 'scrapChampionship', 'cannot set a row with mismatched columns']
['World', 'Trofeo Colombino', '2007', '/soccer/world/trofeo-colombino-2007/', 'scrapChampionship', 'cannot set a row with mismatched columns']
['World', 'Vodacom Challenge', '2007', '/soccer/world/vodacom-challenge-2007/', 'scrapChampionship', 'cannot set a row with mismatched columns']
['World',

In [8]:
row

['Sweden', 'Super Cup', '2014', '/soccer/sweden/super-cup-2014/', 'Timeout']

In [7]:
%%time
#célula de testes
'''page_link = '/soccer/andorra/primera-divisio/results'
page_content = get_page(page_link)
print('Page')
div = page_content.find_all('ul', attrs = {'class' : 'list-tabs list-tabs--secondary'})
if len(div) > 0:
    print('Group')
    #df, df_goals = scrapGroupPhase(div, page_link)
    df = scrapGroupPhase(div, page_link, 'Primeira Liga', '2017')
else:
    print('Champ')
    #df, df_goals = scrapChampionship(page_content)
    df = scrapChampionship(page_content, 'Primeira Liga', '2017')'''

Wall time: 0 ns


"page_link = '/soccer/andorra/primera-divisio/results'\npage_content = get_page(page_link)\nprint('Page')\ndiv = page_content.find_all('ul', attrs = {'class' : 'list-tabs list-tabs--secondary'})\nif len(div) > 0:\n    print('Group')\n    #df, df_goals = scrapGroupPhase(div, page_link)\n    df = scrapGroupPhase(div, page_link, 'Primeira Liga', '2017')\nelse:\n    print('Champ')\n    #df, df_goals = scrapChampionship(page_content)\n    df = scrapChampionship(page_content, 'Primeira Liga', '2017')"